Rozważmy teraz przykłady w których chcemy wyciągnąć dane o ścieżkach dowolnej długości.

In [1]:
USE graphdemo
GO

Commands completed successfully.

Total execution time: 00:00:00.016

Rozpocznijmy od zwykłego zapytania grafowego

In [22]:
SELECT man1.name, man2.name
FROM Person man1, friendOf, Person man2
WHERE MATCH(man1-(friendOf)->man2)
AND man1.name = 'John'

(1 row affected)

Total execution time: 00:00:00.072

name,name
John,Mary


Chcielibyśmy teraz powielić dowolnie ilość krawędzi na ścieżce do wierzchołka końcowego, tak aby zapytanie reprezentowało ścieżkę dowolnej długości.

Pierwsze (intuicyjne) podejście: dodanie operatora plusa...

In [23]:
SELECT man1.name, man2.name
FROM Person man1, friendOf, Person man2
WHERE MATCH(man1(-(friendOf)->man2)+)
AND man1.name = 'John'

: Msg 102, Level 15, State 1, Line 3
Incorrect syntax near '('.

Total execution time: 00:00:00.002

...nie zadziała. Wzorce zmiennej długości muszą być dodatkowo otoczone funkcją SHORTEST\_PATH...

In [4]:
SELECT man1.name, man2.name
FROM Person man1, friendOf, Person man2
WHERE MATCH(SHORTEST_PATH(man1(-(friendOf)->man2)+))
AND man1.name = 'John'

: Msg 13948, Level 16, State 1, Line 1
The table name or alias 'friendOf' must be marked as FOR PATH to be used in the recursive section of a SHORTEST_PATH clause.

: Msg 13948, Level 16, State 1, Line 1
The table name or alias 'man2' must be marked as FOR PATH to be used in the recursive section of a SHORTEST_PATH clause.

Total execution time: 00:00:00.012

...a tabele wzięte do części rekurencyjnej wzorca muszą być oznaczone słowami kluczowymi `FOR PATH`

In [5]:
SELECT man1.name, man2.name
FROM Person            AS man1
   , friendOf FOR PATH AS fo
   , Person   FOR PATH AS man2
WHERE MATCH(SHORTEST_PATH(man1(-(fo)->man2)+))
AND man1.name = 'John'

: Msg 13961, Level 16, State 1, Line 1
The alias or identifier 'man2.name' cannot be used in the select list, order by, group by, or having context.

Total execution time: 00:00:00.053

Na razie pomińmy również `man2.name` i zobaczmy wynik zapytania.

In [7]:
SELECT man1.name
FROM Person AS man1
   , friendOf FOR PATH AS fo
   , Person FOR PATH AS man2
WHERE MATCH(SHORTEST_PATH(man1(-(fo)->man2)+))
AND man1.name = 'John'

(3 rows affected)

Total execution time: 00:00:00.043

name
John
John
John


Otrzymujemy w wyniku trzy rekordy. Odpowiadają one ścieżkom rozpoczynającym się od węzła "John" i kończącym w innych węzłach, w tym przypadku z tabeli Person. W sekcji SELECT nie można bezpośrednio odwołać się do tabel ze ścieżki, gdyż mogą się one potencjalnie odwoływać do wielu rekordów na ścieżce. Z tego też powodu rekordy FOR PATH są zbierane do grup, do których należy zaaplikować odpowiednie funkcje agregacji (STRING_AGG, LAST_VALUE, COUNT, SUM, AVG, MIN, MAX), po których musi nastąpić wyrażenie "WITHIN GROUP (GRAPH PATH)"

In [8]:
SELECT man1.name                                              AS FirstNode
     , LAST_VALUE(man2.name)        WITHIN GROUP (GRAPH PATH) AS LastNode
     , STRING_AGG(man2.name, '->')  WITHIN GROUP (GRAPH PATH) AS Path
     , COUNT(man2.name)             WITHIN GROUP (GRAPH PATH) AS PathLength
     , SUM(fo.relationshipStrength) WITHIN GROUP (GRAPH PATH) AS RelationshipStrengthDistance
     , AVG(fo.relationshipStrength) WITHIN GROUP (GRAPH PATH) AS AverageRelationshipStrength
     , MIN(fo.relationshipStrength) WITHIN GROUP (GRAPH PATH) AS MinimumRelationshipStrength
     , MAX(fo.relationshipStrength) WITHIN GROUP (GRAPH PATH) AS MaximumRelationshipStrength
FROM Person            AS man1
   , friendOf FOR PATH AS fo
   , Person   FOR PATH AS man2
WHERE MATCH(SHORTEST_PATH(man1(-(fo)->man2)+))
AND man1.name = 'John'

(3 rows affected)

Total execution time: 00:00:00.075

FirstNode,LastNode,Path,PathLength,RelationshipStrengthDistance,AverageRelationshipStrength,MinimumRelationshipStrength,MaximumRelationshipStrength
John,Mary,Mary,1,"1,100000023841858","1,100000023841858","1,1","1,1"
John,Alice,Mary->Alice,2,"3,600000023841858","1,800000011920929","1,1","2,5"
John,John,Mary->Alice->John,3,"3,900000035762787","1,300000011920929","0,3","2,5"


Aby dodać ograniczenia dotyczące którejkolwiek z tych wartości należy użyć zapytania okalającego

In [9]:
SELECT * FROM (
SELECT man1.name                                              AS FirstNode
     , LAST_VALUE(man2.name)        WITHIN GROUP (GRAPH PATH) AS LastNode
     , STRING_AGG(man2.name, '->')  WITHIN GROUP (GRAPH PATH) AS Path
     , COUNT(man2.name)             WITHIN GROUP (GRAPH PATH) AS PathLength
     , SUM(fo.relationshipStrength) WITHIN GROUP (GRAPH PATH) AS RelationshipStrengthDistance
     , AVG(fo.relationshipStrength) WITHIN GROUP (GRAPH PATH) AS AverageRelationshipStrength
     , MIN(fo.relationshipStrength) WITHIN GROUP (GRAPH PATH) AS MinimumRelationshipStrength
     , MAX(fo.relationshipStrength) WITHIN GROUP (GRAPH PATH) AS MaximumRelationshipStrength
FROM Person            AS man1
   , friendOf FOR PATH AS fo
   , Person   FOR PATH AS man2
WHERE MATCH(SHORTEST_PATH(man1(-(fo)->man2)+))
AND man1.name = 'John'
) Q
WHERE Q.LastNode = 'Alice'

(1 row affected)

Total execution time: 00:00:00.075

FirstNode,LastNode,Path,PathLength,RelationshipStrengthDistance,AverageRelationshipStrength,MinimumRelationshipStrength,MaximumRelationshipStrength
John,Alice,Mary->Alice,2,"3,600000023841858","1,800000011920929","1,1","2,5"


Inny przykład: znajdź tylko takie łańcuszki, gdzie siła przyjaźni nie spada poniżej poziomu 1.0

In [24]:
SELECT * FROM (
SELECT (man1.name + '->' + STRING_AGG(man2.name, '->')  WITHIN GROUP (GRAPH PATH)) AS Path
     ,                     MIN(fo.relationshipStrength) WITHIN GROUP (GRAPH PATH)  AS MinimumRelationshipStrength
FROM Person            AS man1
   , friendOf FOR PATH AS fo
   , Person   FOR PATH AS man2
WHERE MATCH(SHORTEST_PATH(man1(-(fo)->man2)+))
) Q
WHERE Q.MinimumRelationshipStrength >= 1.0

(5 rows affected)

Total execution time: 00:00:00.112

Path,MinimumRelationshipStrength
John->Mary,"1,1"
Mary->Alice,"2,5"
Jacob->Mary,"1,4"
John->Mary->Alice,"1,1"
Jacob->Mary->Alice,"1,4"


W kolejnych przykładach posłużymy się schematem `klient-\>restauracja\<-klient-\>restauracja-\>...`, aby znaleźć potencjalnie podobne restauracje (załóżmy, że takiej heurystyki chcemy użyć)

In [25]:
SELECT man.name, likes.rating, rest.name
FROM Person man
   , Restaurant rest
   , likes
WHERE MATCH(man-(likes)->rest)
OR MATCH()

(6 rows affected)

Total execution time: 00:00:00.033

name,rating,name
John,9,Taco Dell
Mary,9,Ginger and Spice
Alice,9,Noodle Land
Jacob,9,Noodle Land
Jacob,9,Ginger and Spice
Julie,9,Noodle Land


In [28]:
SELECT * FROM (
SELECT man1.name AS FirstNode
     , STRING_AGG(N'-'+CONVERT(VARCHAR(4), l1.rating)+'->['+rest.name+']<-'+CONVERT(VARCHAR(4), l2.rating)+'-'+man2.name, '') WITHIN GROUP (GRAPH PATH) AS Path
     , LAST_VALUE(man2.name) WITHIN GROUP (GRAPH PATH) as LastNode
FROM Person AS man1
   , likes FOR PATH AS l1
   , Restaurant FOR PATH AS rest
   , likes FOR PATH AS l2
   , Person FOR PATH as man2 
WHERE MATCH(SHORTEST_PATH(man1(-(l1)->rest<-(l2)-man2){1,4}))
) Q WHERE Q.FirstNode <> Q.LastNode

(6 rows affected)

Total execution time: 00:00:00.211

FirstNode,Path,LastNode
Jacob,-9->[Noodle Land]<-9-Alice,Alice
Julie,-9->[Noodle Land]<-9-Alice,Alice
Jacob,-9->[Ginger and Spice]<-9-Mary,Mary
Jacob,-9->[Noodle Land]<-9-Alice,Alice
Julie,-9->[Noodle Land]<-9-Alice,Alice
Mary,-9->[Ginger and Spice]<-9-Jacob,Jacob
Alice,-9->[Noodle Land]<-9-Jacob,Jacob
Julie,-9->[Noodle Land]<-9-Jacob,Jacob
Alice,-9->[Noodle Land]<-9-Julie,Julie
Jacob,-9->[Noodle Land]<-9-Julie,Julie


W czterech ostatnich wierszach mamy krawędź pomiędzy restauracją Taco Dell a Jacobem, która nie powinna się tam znaleźć, jako że nie ma takiej w bazie (możemy to sprawdzić powyżej). Widzimy stąd, że tworzenie bardziej złożonych wzorców niż rekursja jednej krawędzi może produkować niepoprawne wyniki.

In [17]:
SELECT rest1.name
     , STRING_AGG(N'-'+CONVERT(VARCHAR(4), l1.rating)+'->['+man.name+']<-'+CONVERT(VARCHAR(4), l2.rating)+'-'+rest2.name, '') WITHIN GROUP (GRAPH PATH) AS Path
FROM Restaurant AS rest1
   , likes FOR PATH AS l1
   , Person FOR PATH AS man
   , likes FOR PATH AS l2
   , Restaurant FOR PATH as rest2 
WHERE MATCH(SHORTEST_PATH(rest1(<-(l1)-man-(l2)->rest2){1,4}))

(3 rows affected)

Total execution time: 00:00:00.085

name,Path
Taco Dell,-9->[John]<-9-Taco Dell
Ginger and Spice,-9->[Mary]<-9-Ginger and Spice
Noodle Land,-9->[Alice]<-9-Noodle Land


 Ograniczenia `SHORTEST_PATH` mogą być łączone z innymi ograniczeniami grafowymi.

W poniższym przykładzie poszukujemy

In [ ]:
SELECT
	Person1.name AS PersonName,
	STRING_AGG(Person2.name, '->') WITHIN GROUP (GRAPH PATH) AS Friends,
	Restaurant.name
FROM
	Person AS Person1,
	friendOf FOR PATH AS fo,
	Person FOR PATH  AS Person2,
	likes,
	Restaurant
WHERE MATCH(SHORTEST_PATH(Person1(-(fo)->Person2){1,3}) AND LAST_NODE(Person2)-(likes)->Restaurant )
AND Person1.name = 'Jacob'
AND Restaurant.name = 'Ginger and Spice'

In [2]:
DROP TABLE IF EXISTS likes;
DROP TABLE IF EXISTS Person;
DROP TABLE IF EXISTS Restaurant;
DROP TABLE IF EXISTS City;
DROP TABLE IF EXISTS friendOf;
DROP TABLE IF EXISTS livesIn;
DROP TABLE IF EXISTS locatedIn;
DROP TABLE IF EXISTS Bank;
DROP TABLE IF EXISTS owesMoney;

Commands completed successfully.

Total execution time: 00:00:00.056